In [1]:
import sys;print(sys.version)

3.10.8 (v3.10.8:aaaf517424, Oct 11 2022, 10:14:40) [Clang 13.0.0 (clang-1300.0.29.30)]


In [1]:
os.getcwd() 

'/Users/biancaziliotto/Desktop/GitRepo/MLx2'

In [1]:
os.chdir('src')

In [2]:
from MLP import MLP 

In [3]:
mlp = MLP(0, [], 3, 2)

In [11]:
mlp.layers

In [12]:
mlp = MLP(1, [1], 3, 2)

TypeError: Layer.__init__() takes 1 positional argument but 4 were given

In [5]:
import numpy as np
X = np.ndarray((4,3))
print(X)
mlp.predict(X)
mlp.fit(X, X[:,1:], batch_size=2, error_function_str="MSE")

[[2.31584178e+077 2.68679196e+154 2.96439388e-323]
 [0.00000000e+000 0.00000000e+000 1.16095484e-028]
 [2.17236968e-153 8.96826103e+130 3.01478101e-110]
 [6.48224637e+170 3.67145870e+228 8.51909448e-309]]


AttributeError: 'MLP' object has no attribute 'batch_size'

2
